<a href="https://colab.research.google.com/github/naafey-aamer/xray_classifier/blob/main/Pneumonia_Classifier_EffiecientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install efficientnet
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
import efficientnet.tfkeras as efn

# Load the pre-trained ResNet-50 model (without the top fully connected layers)
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# replacing ResNet50 with EfficientNetB0
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.5 MB/s eta 0:00:00
16804768/16804768 [==============================] - 0s 0us/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define data directories for train, validation, and test sets
train_data_dir = '/content/drive/MyDrive/xray_pneu/train'
val_data_dir = '/content/drive/MyDrive/xray_pneu/val'
test_data_dir = '/content/drive/MyDrive/xray_pneu/test'

# Set up data generators for train, validation, and test sets
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']  # Classes should be specified based on subfolder names
)

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(224, 224),
    batch_size=4,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary',
    classes=['NORMAL', 'PNEUMONIA']
)

Mounted at /content/drive
Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
# print(train_generator[])

In [ ]:
# Freeze pre-trained layers
# for layer in base_model.layers:
#     layer.trainable = False

# Unfreeze all layers
for layer in base_model.layers:
    layer.trainable = True

# print(len(base_model.layers))

# keep the bottom layers (first 170) frozen
for layer in base_model.layers[:220]:
    layer.trainable = False

# new dense layers for binary classification
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.7)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# new model with the added layers
model = tf.keras.Model(inputs=base_model.input, outputs=output)
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=0.001, momentum=0.9),
    metrics=['accuracy']
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn (BatchNormalization)   (None, 112, 112, 32  128         ['stem_conv[0][0]']              
                                )                                                             

In [ ]:
def lr_schedule(epoch, lr):
    # Reduce learning rate by 10% every 2 epochs
    if (epoch+1) % 2 == 0:
        lr *= 0.9
    return max(lr, 3e-4)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=1, verbose=1)

# Fit the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=6,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=[lr_callback, earlystop]
)


Epoch 1: LearningRateScheduler setting learning rate to 0.009999999776482582.
Epoch 1/6
41/41 [==============================] - 545s 13s/step - loss: 0.3484 - accuracy: 0.8338 - val_loss: 1.1598 - val_accuracy: 0.5000 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.008999999798834325.
Epoch 2/6
41/41 [==============================] - 497s 12s/step - loss: 0.1502 - accuracy: 0.9427 - val_loss: 0.7466 - val_accuracy: 0.6250 - lr: 0.0090

Epoch 3: LearningRateScheduler setting learning rate to 0.008999999612569809.
Epoch 3/6
41/41 [==============================] - 498s 12s/step - loss: 0.1258 - accuracy: 0.9509 - val_loss: 0.4325 - val_accuracy: 0.8125 - lr: 0.0090

Epoch 4: LearningRateScheduler setting learning rate to 0.008099999651312828.
Epoch 4/6
41/41 [==============================] - 484s 12s/step - loss: 0.1066 - accuracy: 0.9630 - val_loss: 0.3079 - val_accuracy: 0.9375 - lr: 0.0081

Epoch 5: LearningRateScheduler setting learning rate to 0.008099999

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test accuracy: {test_accuracy}')

10/10 [==============================] - 122s 13s/step - loss: 0.4394 - accuracy: 0.8429
Test accuracy: 0.8429487347602844
